# Analyse et visualisation de données avec Python
## Accéder à des bases de données SQLite via Pandas
Questions
* Si mes données sont dans une base de données SQL, puis-je les utiliser?
* Comment sauvegarder des données de Python à une base de données SQL?

Objectifs
* Utiliser le module `sqlite3` pour interagir avec une base de données SQL.
* Accéder aux données d'un fichier SQLite via Python.
* Décrire les différences entre utiliser un fichier CSV et un fichier SQLite.
* Décrire les avantages à utiliser une base de données SQL.

## CSV vs SQLite
Stocker les données dans un fichier SQLite peut accélérer substantiellement les opérations de lecture et écriture en comparaison avec un fichier CSV. La différence en performance devient de plus en plus évidente lorsque le jeu de données grossit (voir par exemple [ce test de performance](http://sebastianraschka.com/Articles/2013_sqlite_database.html#results-and-conclusions)).

## Python et SQL
### Le module `sqlite3`

In [ ]:
import sqlite3

In [ ]:
# Créer une connexion SQL à notre fichier SQLite
con = sqlite3.connect("../data/portal_mammals.sqlite")

cur = con.cursor()

# Le résultat de "cursor.execute()" peut être traité rangée par rangée
for rangee in cur.execute('SELECT * FROM species;'):
    print(rangee)

# Assurez-vous de fermer la connexion
con.close()

### Différentes requêtes

In [ ]:
# Créer une connexion SQL à notre fichier SQLite
con = sqlite3.connect("../data/portal_mammals.sqlite")

cur = con.cursor()

In [ ]:
# Obtenir la liste de toutes les tables
cur.execute('SELECT name FROM sqlite_master WHERE type = "table"')
cur.fetchall()

In [ ]:
# Obtenir les informations de la table "plots" (sites)
cur.execute('PRAGMA table_info(plots)')
cur.fetchall()

In [ ]:
# Obtenir les différents types de sites et leur décompte
# Le tout sera trié selon la valeur de décompte
cur.execute('SELECT plot_type,COUNT(plot_type) FROM plots ' +
            'GROUP BY plot_type ' +
            'ORDER BY COUNT(plot_type)')
cur.fetchall()

In [ ]:
# Obtenir tous les identifiants de sites lorsque plot_type est "Spectab exclosure"
cur.execute('SELECT plot_id FROM plots WHERE plot_type = "Spectab exclosure"')
cur.fetchall()

In [ ]:
# Obtenir seulement une ligne de résultat d'une requête
cur.execute('SELECT genus,COUNT(genus) FROM surveys ' +
            'INNER JOIN plots   ON plots.plot_id      = surveys.plot ' +
            'INNER JOIN species ON species.species_id = surveys.species ' +
            'GROUP BY genus HAVING plots.plot_type = "Spectab exclosure" ' +
            'ORDER BY COUNT(genus) DESC')
cur.fetchone()

In [ ]:
# Assurez-vous de fermer la connexion
con.close()

### Exercice - Requêtes SQL
Créez une requête contenant les données enregistrées entre 1998 et 2001, pour les observations de sexe "F" ou "M". Gardez les différentes combinaisons de `genus`, nom d'espèce et `plot_type`. Combien d'enregistrements avons-nous à la fin?

In [ ]:
con = sqlite3.connect("../data/portal_mammals.sqlite")
cur = con.cursor()

cur.execute('SELECT DISTINCT genus,species.species,plot_type FROM surveys ' +
            'INNER JOIN plots   ON plots.plot_id      = surveys.plot ' +
            'INNER JOIN species ON species.species_id = surveys.species ' +
            'WHERE (year BETWEEN 1998 AND 2001) ' +
            'AND (sex IN ("F", "M"))')
print(len(cur.fetchall()))

con.close()

## Charger les données d'une table SQLite dans un DataFrame

In [ ]:
import pandas as pd

con = sqlite3.connect("../data/portal_mammals.sqlite")
df = pd.read_sql_query("SELECT * FROM surveys", con)

print(df.head())

con.close()

### Exercice - SQL et Pandas
Créez un dataframe contenant le nombre total d'observations (`COUNT`) et la somme (`SUM`) des poids (`wgt`) pour chaque identifiant de site (`plot`) et pour chaque année, le tout trié par identifiant de site.

In [ ]:
con = sqlite3.connect("../data/portal_mammals.sqlite")

df = pd.read_sql_query('SELECT plot,year,COUNT(record_id),SUM(wgt) FROM surveys ' +
                       'GROUP BY plot,year ORDER BY plot', con)

print(df.head())
print(df.tail())

con.close()

## Sauvegarder les données - créer de nouvelles tables

In [ ]:
con = sqlite3.connect("../data/portal_mammals.sqlite")

# Charger les données dans un DataFrame
surveys_df = pd.read_sql_query("SELECT * FROM surveys", con)

# Sélectionner les données pour l'année 2002
surveys2002 = surveys_df[surveys_df['year'] == 2002]

# Sauvegarder le nouveau DataFrame dans une nouvelle table SQL
surveys2002.to_sql("surveys2002", con, if_exists="replace")

con.close()